In [12]:
# We need a higher version of scikit-learn than the pre-installed one
!sudo pip install scikit-learn==0.20.1
# restart the notebook after installation

In [10]:
from google.cloud import storage
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.externals import joblib

In [2]:
# specify bucket name
BUCKET_NAME = "test_34336"

# client and bucket
client = storage.client.Client()
bucket = client.bucket(BUCKET_NAME)

# Data exploration

In [3]:
# read data
data = pd.read_csv("titanic.csv")

In [7]:
# have a look at first several lines
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
# describe statistics of data
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


# Data Preprocessing

In [4]:
# fill nan will null
X = data[["Pclass", "Sex", "SibSp", "Parch"]]
X = X.fillna("null")

In [5]:
y = data[["Survived"]]

In [6]:
# one-hot encoding
encoder = OneHotEncoder()
encoder.fit(X)
X_data = encoder.transform(X)

In [7]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X_data, y)

# Model construction

In [8]:
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X_train, y_train)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

In [9]:
model.score(X_test, y_test)

0.7937219730941704

# Save model and Upload model to Cloud Storage for deployment

In [12]:
joblib.dump(model, "titanic_model.joblib")
joblib.dump(encoder, "titanic_encoder.joblib")

['titanic_encoder.joblib']

In [13]:
# upload files to Cloud storage
blob = bucket.blob("titanic_model.joblib")
blob.upload_from_string("titanic_model.joblib")

blob = bucket.blob("titanic_encoder.joblib")
blob.upload_from_string("titanic_encoder.joblib")